In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [2]:
import os
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'

os.environ['PYSPARK_PYTHON']='python'

In [3]:
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "4g").appName("test2").getOrCreate()

In [4]:
mvcdata = spark.read.format('csv').options(header='true',inferschema='true').load("learningData.csv")
mvcdata.createOrReplaceTempView("allData")

In [7]:
null_modified_sql = "SELECT get_borough(a.LATITUDE,a.LONGITUDE,a.BOROUGH) as B1, a.* from allData a " 

In [8]:
from shapely.geometry import Point, shape
import geopandas as gpd
def get_borough(latitude, longitude,borough):
    if borough is not None:
        return borough
    if latitude is None or longitude is None:
        return ""
    shapefile_path = 'nyc_borough.shx'
    boroughs_gdf = gpd.read_file(shapefile_path)
    point = Point(longitude, latitude)
    boroughs = ["BRONX","QUEENS","MANHATTAN","BROOKLYN","STATEN ISLAND"]
    for index, row in boroughs_gdf.iterrows():
        polygon = row['geometry']
        if polygon.contains(point):
            #print(boroughs[index])
            return boroughs[index]
            #return row['Borough']

    return ''

In [9]:
spark.udf.register("get_borough",get_borough)

<function __main__.get_borough(latitude, longitude, borough)>

In [10]:
null_modified_df = spark.sql(null_modified_sql)

In [12]:
null_modified_df.show()

+-------------+----------+----------+---------+--------+---------+----------+--------------------+--------------------+--------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|           B1|CRASH DATE|CRASH TIME|  BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|   CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF 

In [15]:
test_query = "SELECT COUNT(*) from testData where B1 is null"

In [16]:
null_modified_df.createOrReplaceTempView("testData")
test_df = spark.sql(test_query)
test_df.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [ ]:
null_modified_df.repartition(1).write.format('csv').options(header='true').save("borough_no_null")